song recommendation system

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from ydata_profiling import ProfileReport


# text preprocessing 
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

## dorp the model in pickle 
import pickle


#  Load the dataset

In [2]:
# Load the dataset (Replace 'songs.csv' with your actual dataset)
song_rec = pd.read_csv('D:\\song-recommendation-system\\data\\spotify_millsongdata.csv',nrows=5000)

In [3]:
song_rec.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
song_rec = song_rec.rename(columns={'text': 'song Lyrics', 'song': 'song_name','artist':'artist_name'})

In [5]:
song_rec.shape 

(5000, 4)

In [6]:
song_rec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  5000 non-null   object
 1   song_name    5000 non-null   object
 2   link         5000 non-null   object
 3   song Lyrics  5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [7]:
total_nulls = song_rec.isnull().sum().sum()
print(f"Total Missing Values: {total_nulls}")


Total Missing Values: 0


In [8]:

# Check for duplicate rows
duplicate_count = song_rec.duplicated().sum()
print(f"Total Duplicate Rows: {duplicate_count}")

Total Duplicate Rows: 0


In [9]:

# Drop a single column
song_rec = song_rec.drop(columns=['link'])


Text preprocessing 

In [10]:
# lower casing 
song_rec['song Lyrics'] = song_rec['song Lyrics'].str.lower()
# song_rec['song_name'] = song_rec['song_name'].str.lower()

In [11]:
song_rec['song Lyrics'][1000]

"here we are, seven days  \r\nand seven nights of empty tries  \r\nit's ritual, habitual  \r\nbut it's never gonna work this time  \r\n  \r\nwe're to the point of no return  \r\nand along the way the only thing we've learned  \r\nis how to hurt each other  \r\n  \r\ni'm looking back and wondering why  \r\nit took so long to realize  \r\nthat nothing's changed, it never will  \r\nall these years of standing still  \r\nand still we stay in all this pain  \r\nand nothing's gonna make it go away  \r\n  \r\ni don't wanna wait another minute  \r\nput me out of my misery  \r\ni can read your mind baby you're not in it  \r\nand we're not what we used to be  \r\nno you wouldn't have to lie to me  \r\nif you would only let me go  \r\nand i don't wanna wait another minute to hear  \r\nsomething that i already know  \r\ni know, i know, i know  \r\nsomething that i already know  \r\ni know, i know, i know  \r\n  \r\nso save your voice  \r\ndon't waste your breath  \r\ncan't you see we're at the end

In [12]:
import string
exclude  = string.punctuation
print(exclude)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [13]:
# Remove punctuation from the 'Text' column
song_rec['song Lyrics'] = song_rec['song Lyrics'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
# song_rec['song_name'] = song_rec['song_name'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [14]:
song_rec.head()

,artist_name,song_name,song Lyrics
0,ABBA,Ahe's My Kind Of Girl,look at her face its a wonderful face \r\nand...
1,ABBA,"Andante, Andante",take it easy with me please \r\ntouch me gent...
2,ABBA,As Good As New,ill never know why i had to go \r\nwhy i had ...
3,ABBA,Bang,making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...


In [15]:
song_rec['song Lyrics']

0       look at her face its a wonderful face  \r\nand...
1       take it easy with me please  \r\ntouch me gent...
2       ill never know why i had to go  \r\nwhy i had ...
3       making somebody happy is a question of give an...
4       making somebody happy is a question of give an...
                              ...                        
4995    you wont take my love for tender  \r\nyou can ...
4996    ive looked at it every way i can  \r\nfrom und...
4997    i wont walk with my head bowed  \r\nbe on beyo...
4998    dressed up like a dogs dinner  \r\nbutter woul...
4999    now theres newsprint all over your face  \r\nw...
Name: song Lyrics, Length: 5000, dtype: object

In [16]:
song_rec['song Lyrics'][0]

'look at her face its a wonderful face  \r\nand it means something special to me  \r\nlook at the way that she smiles when she sees me  \r\nhow lucky can one fellow be  \r\n  \r\nshes just my kind of girl she makes me feel fine  \r\nwho could ever believe that she could be mine  \r\nshes just my kind of girl without her im blue  \r\nand if she ever leaves me what could i do what could i do  \r\n  \r\nand when we go for a walk in the park  \r\nand she holds me and squeezes my hand  \r\nwell go on walking for hours and talking  \r\nabout all the things that we plan  \r\n  \r\nshes just my kind of girl she makes me feel fine  \r\nwho could ever believe that she could be mine  \r\nshes just my kind of girl without her im blue  \r\nand if she ever leaves me what could i do what could i do\r\n\r\n'

In [17]:
# Function to clean spaces and newlines
def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()  # Replaces multiple spaces/newlines with a single space

# Apply function to the column
song_rec['song Lyrics'] = song_rec['song Lyrics'].apply(clean_text)

In [18]:
song_rec['song Lyrics'][0]

'look at her face its a wonderful face and it means something special to me look at the way that she smiles when she sees me how lucky can one fellow be shes just my kind of girl she makes me feel fine who could ever believe that she could be mine shes just my kind of girl without her im blue and if she ever leaves me what could i do what could i do and when we go for a walk in the park and she holds me and squeezes my hand well go on walking for hours and talking about all the things that we plan shes just my kind of girl she makes me feel fine who could ever believe that she could be mine shes just my kind of girl without her im blue and if she ever leaves me what could i do what could i do'

In [19]:
song_rec['song Lyrics'][13]

'youre so hot teasing me so youre blue but i cant take a chance on a chick like you thats something i couldnt do theres that look in your eyes i can read in your face that your feelings are driving you wild ah but girl youre only a child well i can dance with you honey if you think its funny does your mother know that youre out and i can chat with you baby flirt a little maybe does your mother know that youre out take it easy take it easy better slow down girl thats no way to go does your mother know take it easy take it easy try to cool it girl take it nice and slow does your mother know i can see what you want but you seem pretty young to be searching for that kind of fun so maybe im not the one now youre so cute i like your style and i know what you mean when you give me a flash of that smile smile but girl youre only a child well i can dance with you honey if you think its funny does your mother know that youre out and i can chat with you baby flirt a little maybe does your mother 

In [20]:
song_rec.head()

,artist_name,song_name,song Lyrics
0,ABBA,Ahe's My Kind Of Girl,look at her face its a wonderful face and it m...
1,ABBA,"Andante, Andante",take it easy with me please touch me gently li...
2,ABBA,As Good As New,ill never know why i had to go why i had to pu...
3,ABBA,Bang,making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...


In [21]:
from nltk.stem import PorterStemmer

# Initialize the PorterStemmer
stemmer = PorterStemmer()

# Function to apply stemming
def apply_stemming(text):
    words = text.split()  # Split text into individual words
    stemmed_words = [stemmer.stem(word) for word in words]  # Apply stemming to each word
    return " ".join(stemmed_words)  # Rejoin words back into a string

# Apply stemming function to the 'reviews' column
song_rec['song Lyrics'] = song_rec['song Lyrics'].apply(apply_stemming)

In [22]:
song_rec

,artist_name,song_name,song Lyrics
0,ABBA,Ahe's My Kind Of Girl,look at her face it a wonder face and it mean ...
1,ABBA,"Andante, Andante",take it easi with me pleas touch me gentli lik...
2,ABBA,As Good As New,ill never know whi i had to go whi i had to pu...
3,ABBA,Bang,make somebodi happi is a question of give and ...
4,ABBA,Bang-A-Boomerang,make somebodi happi is a question of give and ...
...,...,...,...
4995,Elvis Costello,Love For Tender,you wont take my love for tender you can put y...
4996,Elvis Costello,Love Went Mad,ive look at it everi way i can from under and ...
4997,Elvis Costello,Lover's Walk,i wont walk with my head bow be on beyond caut...
4998,Elvis Costello,Luxembourg,dress up like a dog dinner butter wouldnt melt...


In [23]:
song_rec['song Lyrics'][0]

'look at her face it a wonder face and it mean someth special to me look at the way that she smile when she see me how lucki can one fellow be she just my kind of girl she make me feel fine who could ever believ that she could be mine she just my kind of girl without her im blue and if she ever leav me what could i do what could i do and when we go for a walk in the park and she hold me and squeez my hand well go on walk for hour and talk about all the thing that we plan she just my kind of girl she make me feel fine who could ever believ that she could be mine she just my kind of girl without her im blue and if she ever leav me what could i do what could i do'

In [24]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features=5000,stop_words='english')

In [25]:
# vector = cv.fit_transform(song_rec['song Lyrics'])

In [26]:
# vector.shape 

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(song_rec['song Lyrics'])
similarity = cosine_similarity(matrix)

In [30]:
def recommendation(song_df):
    # Check if the song exists in the dataset
    if song_df not in song_rec['song_name'].values:
        return f"Error: '{song_df}' not found in the dataset."

    idx = song_rec[song_rec['song_name'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    songs = []
    for m_id in distances[1:21]:  # Get top 20 recommendations
        songs.append(song_rec.iloc[m_id[0]]['song_name'])  # Fix column reference

    return songs

# Test the function
print(recommendation('As Good As New'))


['Ma Baker', "Ma' Dear Ma' Dear", 'Juice', 'It Had To Be You', 'Come Home Johnny Birdie', 'Je Ne Vous Oublie Pas', 'Monday', 'New Shoes', 'Spaced', 'At My Window', 'Feel Like A Number', 'On Fire', "Di Fronte All'amore", 'Good Enough', 'I Feel Good', 'Gotta Move', 'Good To My Baby', 'Brand New', 'Five Years', 'Feel Good']


In [31]:
(recommendation('As Good As New'))

['Ma Baker',
 "Ma' Dear Ma' Dear",
 'Juice',
 'It Had To Be You',
 'Come Home Johnny Birdie',
 'Je Ne Vous Oublie Pas',
 'Monday',
 'New Shoes',
 'Spaced',
 'At My Window',
 'Feel Like A Number',
 'On Fire',
 "Di Fronte All'amore",
 'Good Enough',
 'I Feel Good',
 'Gotta Move',
 'Good To My Baby',
 'Brand New',
 'Five Years',
 'Feel Good']

In [32]:
import pickle
import os 

In [33]:
%pwd

'd:\\song-recommendation-system\\notebook'

In [34]:
os.chdir("../")
%pwd

'd:\\song-recommendation-system'

In [35]:
import os
import pickle

# Create 'model' directory if it doesn't exist
os.makedirs('model', exist_ok=True)

# Save the files inside the 'model' directory
pickle.dump(song_rec, open('model/song_list.pkl', 'wb'))
pickle.dump(similarity, open('model/similarity.pkl', 'wb'))

print("Pickle files saved successfully in 'model' directory.")

Pickle files saved successfully in 'model' directory.
